In [ ]:
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Recommenders.Neural.MultVAE_PyTorch_Recommender import MultVAERecommender_PyTorch
import numpy as np
import random
import os
import optuna



SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

import scipy.sparse as sps

URM_train = sps.load_npz('Dataset/URM_train.npz')
URM_test_complete = sps.load_npz('Dataset/URM_test.npz')


URM_all= URM_test_complete + URM_train

In [ ]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

def objective_function(trial):
    early_stopping_params = {
        "validation_every_n": 1,
        "stop_on_validation": True,
        "evaluator_object": evaluator,
        "lower_validations_allowed": 10,
        "validation_metric": "MAP",
        "epochs": 500
    }


    n_layers = trial.suggest_categorical('n_layers', [1, 2])
    layer_1 = trial.suggest_int('layer_1', 20, 500, log=True)
    p_dims = [layer_1]
    if n_layers == 2:
        layer_2 = trial.suggest_int('layer_2', layer_1, 2000, log=True)
        p_dims.append(layer_2)
    p_dims.append(URM_train.shape[1])


    learning_rate = trial.suggest_float('learning_rate', 1e-6, 1e-1, log=True)
    batch_size = trial.suggest_categorical('batch_size', [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024])
    dropout = trial.suggest_float('dropout', 0.1, 0.7)
    l2_reg = trial.suggest_float('l2_reg', 1e-7, 0.1, log=True)
    anneal_cap = trial.suggest_float('anneal_cap', 1e-2, 1, log=True)
    total_anneal_steps = trial.suggest_int('total_anneal_steps', 1e3, 1e6, log=True)
    sgd_mode = trial.suggest_categorical('sgd_mode', ['adam', 'sgd', 'adagrad'])  # Scelta tra ottimizzatori disponibili


    recommender_instance = MultVAERecommender_PyTorch(URM_train=URM_train)
    recommender_instance.fit(
        learning_rate=learning_rate,
        batch_size=batch_size,
        dropout=dropout,
        total_anneal_steps=total_anneal_steps,
        anneal_cap=anneal_cap,
        p_dims=p_dims,
        l2_reg=l2_reg,
        sgd_mode=sgd_mode,
        **early_stopping_params
    )

    # Recupera il numero di epoche effettivamente eseguite con early stopping
    epochs = recommender_instance.get_early_stopping_final_epochs_dict()["epochs"]
    trial.set_user_attr("epochs", epochs)

    # Valuta il modello
    result_df, _ = evaluator.evaluateRecommender(recommender_instance)

    # Ritorna il MAP (Mean Average Precision) a cutoff 10
    return result_df.loc[10, "MAP"]


In [ ]:
database_path='hypertuning_results/tuning_MultVAE.db'

from optuna.samplers import RandomSampler

study = optuna.load_study( study_name='hyperparameters_MultVAE',
                                storage=f'sqlite:///{database_path}'

)
study.sampler=RandomSampler(seed=SEED)

study.optimize(objective_function, n_trials= 40,n_jobs=4)